# Solving Problems By Searching

## Single-state problems

A problem is defined by the following

1. The initial state e.g $\text{in A}$
2. The actions/successor function e.g. $S(\text{A}) = \{ \langle \text{A} \rightarrow \text{Z}, \text{Z} \rangle, ... \}$
3. Goal test e.g. $x = \text{in B}$
4. Path cost e.g. the sum of distances, number of actions executed etc. $c(x, a, y)$ is the cost of taking action $a$ in state $x$ and reaching state $y$.

In [50]:
def S(state):
    if   state == "A": return ["S", "T", "Z"]
    elif state == "B": return ["P", "F", "U", "G"]
    elif state == "C": return ["D", "R", "P"]
    elif state == "D": return ["M", "C"]
    elif state == "E": return ["H"]
    elif state == "F": return ["S", "B"]
    elif state == "G": return ["B"]
    elif state == "H": return ["U", "E"]
    elif state == "I": return ["N", "V"]
    elif state == "L": return ["T", "M"]
    elif state == "M": return ["L", "D"]
    elif state == "N": return ["I"]
    elif state == "O": return ["Z", "S"]
    elif state == "P": return ["R", "C", "B"]
    elif state == "R": return ["S", "P", "C"]
    elif state == "S": return ["A", "F", "O", "R"]
    elif state == "T": return ["A", "L"]
    elif state == "U": return ["B", "V", "H"]
    elif state == "V": return ["U", "I"]
    elif state == "Z": return ["A", "O"]
    
# Their is only one "action" to move between states so no need for `a` parameter
def c(x, y):
    def inner(x, y):
        if   x == "A" and y == "S": return 140
        elif x == "A" and y == "T": return 118
        elif x == "A" and y == "Z": return 75
        elif x == "B" and y == "P": return 101
        elif x == "B" and y == "F": return 211
        elif x == "B" and y == "U": return 85
        elif x == "B" and y == "G": return 90
        elif x == "C" and y == "D": return 120
        elif x == "C" and y == "R": return 146
        elif x == "C" and y == "P": return 138
        elif x == "D" and y == "M": return 75
        elif x == "E" and y == "H": return 86
        elif x == "F" and y == "S": return 99
        elif x == "H" and y == "U": return 98
        elif x == "I" and y == "N": return 87
        elif x == "I" and y == "V": return 92
        elif x == "L" and y == "T": return 111
        elif x == "L" and y == "M": return 70
        elif x == "O" and y == "Z": return 71
        elif x == "O" and y == "S": return 151
        elif x == "P" and y == "R": return 97
        elif x == "R" and y == "S": return 80
        elif x == "U" and y == "V": return 142
    
    # ignore order of x and y since cost is symetric
    if x == y: return 0
    else:
        cost = inner(x, y)
        if cost == None:
            cost = inner(y, x)
        return cost
    
def goal_test(state):
    return state == "B"
    
problem = {
    "INITIAL_STATE": "A",
    "S": S,
    "c": c,
    "goal_test": goal_test,
}

# Print the solution with arrows
def str_solution(node):
    def str_route(node):
        (state, parent, _) = node
        if parent is None:
            return state
        else:
            return str_route(parent) + " -> " + state
    
    (_, _, total_cost) = node
    return str_route(node) + " (" + str(total_cost) + ")"

## Tree search
One way to search for a solution to a problem is by forming a search tree starting from the inital state and expanding the tree by apply all the legal actions. We repeatedly do this till the goal state is expanded, thus a solution is found. The _frountier_ is a set of states that have yet to be expanded. The most basic of search algorithums is tree search:

```
functon TREE-SEARCH(problem) returns a solution or failure
    initialize the frountier using the initial state of problem
    loop do
        if the frontier is empty then return faliure
        choose a leaf node and remove if from the frontier
        if the node contains a goal state then return the corresponding solution
        expand the chosen node, adding the resulting nodes to the frontier
```

In search algorithums we make the distinction between states and nodes. States refer to a configuration of the world, nodes are a bookkeeping structure.

In [51]:
def tree_search(problem):
    
    # Initalize frontier list
    frontier = [(problem["INITIAL_STATE"], None, 0)]
    
    while True:
        # frontier is empty, return faliure
        if len(frontier) == 0: 
            return None
        
        # pop a leaf node
        node = frontier.pop(0)
        (state, parent, cost) = node
        
        # if the node is a goal state, return solution
        if problem["goal_test"](state) == True: 
            return node
        
        # expand chosen node
        print("Expanding " + state + "...")
        if problem["S"](state) is not None:
            for next_state in problem["S"](state):
                total_cost = cost + problem["c"](state, next_state)
                new_node = (next_state, node, total_cost)
                frontier.append(new_node)

node = tree_search(problem)
print()
print(str_solution(node))

Expanding A...
Expanding S...
Expanding T...
Expanding Z...
Expanding A...
Expanding F...
Expanding O...
Expanding R...
Expanding A...
Expanding L...
Expanding A...
Expanding O...
Expanding S...
Expanding T...
Expanding Z...
Expanding S...

A -> S -> F -> B (450)


## Graph search

Notice how "A" is expanded multiple times, this is due to _repeated states_ in the search tree. This causes a _loopy path_ in the search tree, which means the search tree is infinite thus the making tractable problems intractable. Graph search fixes this problem by remebering which nodes were visited.

```
function GRAPH-SEARCH(problem) returns a solution, or failure
    initialize the frountier using the initial state of problem
    initialize the explored set to be empty
    loop do
        if the frontier is empty then return faliure
        choose a leaf node and remove if from the frontier
        if the node contains a goal state then return the corresponding solution
        add the node to the explored set
        expand the chosen node, adding the resulting nodes to the frontier
            only if not in the frontier or explored set
```

In [54]:
def graph_search(problem):
        
    # Initalize frontier list
    frontier = [(problem["INITIAL_STATE"], None, 0)]
    explored = set()
    
    while True:
        # frontier is empty, return faliure
        if len(frontier) == 0: 
            return None
        
        # pop a leaf node
        node = frontier.pop(0)
        (state, parent, cost) = node
        
        # if the node is a goal state, return solution
        if problem["goal_test"](state) == True: 
            return node
        
        # add node to explored set
        explored.add(state)
        
        # expand chosen node
        print("Expanding " + state + "...")
        if problem["S"](state) is not None:
            for next_state in problem["S"](state):
                total_cost = cost + problem["c"](state, next_state)
                new_node = (next_state, node, total_cost)
                if next_state not in explored:
                    frontier.append(new_node)

node = graph_search(problem)
print()
print(str_solution(node))

Expanding A...
Expanding S...
Expanding T...
Expanding Z...
Expanding F...
Expanding O...
Expanding R...
Expanding L...
Expanding O...

A -> S -> F -> B (450)


## Measuring problem-solving performance

We can evaluate an algorithums performance in four ways:

1. _Completeness_: Is the algorithum guaranteed to find a solution
2. _Optimality_: Does the strategy find the optimal solution
3. _Time complexity_: How long does it take to find a solution
4. _Space complexity_: How much memory is needed to perform a search

We will use three variables

1. $b$ the branching factor 
2. $d$ depth of the shallowest goal node
3. $m$ maximum length of any path in state space

## Breadth-first search

A search were all nodes at a given depth are expanded, before the next level of nodes are expanded. This is acheived with a FIFO queue as in the graph search implementation above.

- Complete: if $b$ is finite
- Time: $b + b^2 + b^3 + ... + b^d = O(b^d)$ ($b$-ary tree of depth $d$)
- Space: $O(b^d)$ (every node in memory)
- Optimal: If cost is 1 (then solution is closest to start node)

## Depth-first search

Expands the deepest unexpanded node. This is achieved with a LIFO queue (stack).

In [55]:
def depth_first(problem):
        
    # Initalize frontier list
    frontier = [(problem["INITIAL_STATE"], None, 0)]
    explored = set()
    
    while True:
        # frontier is empty, return faliure
        if len(frontier) == 0: 
            return None
        
        # pop a leaf node
        node = frontier.pop()
        (state, parent, cost) = node
        
        # if the node is a goal state, return solution
        if problem["goal_test"](state) == True: 
            return node
        
        # add node to explored set
        explored.add(state)
        
        # expand chosen node
        print("Expanding " + state + "...")
        if problem["S"](state) is not None:
            for next_state in problem["S"](state):
                total_cost = cost + problem["c"](state, next_state)
                new_node = (next_state, node, total_cost)
                if next_state not in explored:
                    frontier.append(new_node)

node = depth_first(problem)
print()
print(str_solution(node))

Expanding A...
Expanding Z...
Expanding O...
Expanding S...
Expanding R...
Expanding C...
Expanding P...

A -> Z -> O -> S -> R -> C -> P -> B (762)


- Complete: in finite spaces with no loops
- Time: $O(b^m)$
- Space: $O(bm)$
- Optimal: No

## Depth-limited search

Depth limited search is a depth-first search with a depth limit $l$.

In [57]:
def depth_limited_search(problem, limit):
    node = (problem["INITIAL_STATE"], None, 0)
    return recursive_dls(node, problem, limit)

def recursive_dls(node, problem, limit):
    (state, parent, cost) = node
    print("Expanding " + state + "...")
    if problem["goal_test"](state):
        return node
    elif limit == 0:
        print("Reached depth limit...")
        return None
    else:
        if problem["S"](state) is not None:
            for next_state in problem["S"](state):
                total_cost = cost + problem["c"](state, next_state)
                child = (next_state, node, total_cost)
                result = recursive_dls(child, problem, limit-1)
                if result != None: 
                    return result
            return None
        
node = depth_limited_search(problem, 5)
print()
print(str_solution(node))

Expanding A...
Expanding S...
Expanding A...
Expanding S...
Expanding A...
Expanding S...
Reached depth limit...
Expanding T...
Reached depth limit...
Expanding Z...
Reached depth limit...
Expanding F...
Expanding S...
Reached depth limit...
Expanding B...

A -> S -> A -> S -> F -> B (730)


## Iterative deepening search

Combines the benifits of breadth-first search with depth-first search by repeatedly calling iterative deepening search with larger limits. This seems costly since we will be doing the same work over and over again, however _most_ nodes are in the last level of the search tree, and are only expanded once, thus its not as costly as one might presume.

In [58]:
def iterative_deepening_search(problem):
    result = None
    depth = 0
    while result == None:
        result = depth_limited_search(problem, depth)
        depth += 1
    return result

node = iterative_deepening_search(problem)
print()
print(str_solution(node))

Expanding A...
Reached depth limit...
Expanding A...
Expanding S...
Reached depth limit...
Expanding T...
Reached depth limit...
Expanding Z...
Reached depth limit...
Expanding A...
Expanding S...
Expanding A...
Reached depth limit...
Expanding F...
Reached depth limit...
Expanding O...
Reached depth limit...
Expanding R...
Reached depth limit...
Expanding T...
Expanding A...
Reached depth limit...
Expanding L...
Reached depth limit...
Expanding Z...
Expanding A...
Reached depth limit...
Expanding O...
Reached depth limit...
Expanding A...
Expanding S...
Expanding A...
Expanding S...
Reached depth limit...
Expanding T...
Reached depth limit...
Expanding Z...
Reached depth limit...
Expanding F...
Expanding S...
Reached depth limit...
Expanding B...

A -> S -> F -> B (450)


- _Complete_: if $b$ is finite
- _Time_: $db + (d-1)b^2 + ... + b^d = O(b^d)$
- _Space_: $O(bd)$
- Optimal: If step cost is 1

## Uniform-cost search

Breadth-first is optimal if the step-costs are equal, however we can extend this to work for any step-cost function. By implementing the frontier as a priority queue we all expand the cheapest cost path first. 

In [85]:
import heapq

# emulates priority queue by removing the lowest costing element
def pop_low(frontier):
    lowest_cost = 1000000
    lowest_indx = None
    for i in range(len(frontier)):
        if frontier[i][2] < lowest_cost:
            lowest_cost = frontier[i][2]
            lowest_indx = i
            
    if i == None:
        return None
    else:
        return frontier.pop(lowest_indx)
    
def uniform_cost_search(problem):
    frontier = [(problem["INITIAL_STATE"], None, 0)]
    explored = set()
    
    while True:
        if len(frontier) == 0: 
            return None
        
        node = pop_low(frontier)        
        (state, parent, cost) = node
        print("Expanding " + state + "...")
        
        if problem["goal_test"](state):
            return node

        explored.add(state)
        
        if problem["S"](state) is not None:
            for next_state in problem["S"](state):
                total_cost = cost + problem["c"](state, next_state)
                child = (next_state, node, total_cost)
                
                # check if next state is in frontier
                def check_frontier(frontier, node):
                    for index, node in enumerate(frontier):
                        (n_state, _, cost) = node
                        if n_state == next_state:
                            return (True, cost, index)
                    return (False, 0, 0)
                        
                (in_frontier, f_cost, f_index) = check_frontier(frontier, node)
                    
                        
                if next_state not in explored and not in_frontier:
                    frontier.append(child)
                elif in_frontier and f_cost > total_cost:
                    print("Replacing ", str_solution(frontier[f_index]), "with", str_solution(child))
                    frontier[f_index] = child
                    
node = uniform_cost_search(problem)
print()
print(str_solution(node))

Expanding A...
Expanding Z...
Expanding T...
Expanding S...
Expanding O...
Expanding R...
Expanding L...
Expanding F...
Expanding M...
Expanding P...
Replacing  A -> S -> F -> B (450) with A -> S -> R -> P -> B (418)
Expanding C...
Expanding D...
Expanding B...

A -> S -> R -> P -> B (418)


- _Complete_: if $b$ is finite and step costs are positive
- _Time_: $O(b^{C^*/\epsilon})$
- _Space_: $O(b^{C^*/\epsilon})$
- Optimal: Yes

Where $C^*$ is the cost of the optimal solution, and every action costs atleast $\epsilon$

## Summary of uninformed algorithums

|          | Breadth-first | Uniform-cost | Depth-first | Depth-limited | Iterative deepening |
| -------- | --- | --- | --- | --- | --- |
| Complete | Yes | Yes | No | No | Yes |
| Time     | $O(b^d)$ | $O(b^{C^*/\epsilon})$ | $O(b^m)$ | $O(b^t)$ | $O(b^d)$ |
| Space    | $O(b^d)$ | $O(b^{C^*/\epsilon})$ | $O(bm)$ | $O(bl)$ | $O(bd)$ |
| Optimal  | Yes | Yes | No | No | Yes |